1. 随机摘选句子 或者 片段 start 规定字数上限
2. 显示原句 用户感觉记住之后 折叠原句
3. 输入回忆结果 然后匹配正确率
4. 加入错题集 和 log.csv

In [1]:
import pandas as pd
import numpy as np
import random
import re
import os 
memories = dict() ## 存储已经记过的信息

file = "早晨从中午开始.txt"
# file = "一只特立独行的猪.txt"
# file = "周杰伦.txt"
# file = "我们为什么会分手.txt"
file = "阳谋高手.txt"
file = "从林肯到奥巴马时代.txt"
file = "原则.txt"
# file = "人类简史.txt"
# file = "金融帝国博弈.txt"


In [2]:
# 去掉txt 中特殊字符 首尾空格 分句
def normal_cut_sentence(text):
    text = re.sub('([。！？\?])([^’”])',r'\1\n\2',text)#普通断句符号且后面没有引号
    text = re.sub('(\.{6})([^’”])',r'\1\n\2',text)#英文省略号且后面没有引号
    text = re.sub('(\…{2})([^’”])',r'\1\n\2',text)#中文省略号且后面没有引号
    text = re.sub('([.。！？\?\.{6}\…{2}][’”])([^’”])',r'\1\n\2',text)#断句号+引号且后面没有引号
    text = text.rstrip()    # 去掉段尾的\n，然后
    texts = text.split("\n")
    texts = [x.strip() for x in texts if x.strip()]
    return texts

# 读取 txt 文件
def read_text(file):
    with open(file, encoding="utf-8") as f:
        content = f.read()
        content = normal_cut_sentence(content)
        return content

texts = read_text(file)
# print(read_text(file))

### 随机选取句子

In [3]:
def select_sentence(texts, paragraphs=1, lower_limit=40, upper_limit=100):
    """_summary_

    Args:
        texts (str): 列表
        paragraphs (int, optional): 选取内容最多多少段. Defaults to 1.
        lower_limit (int, optional): 选取内容至少多少字. Defaults to 40.
        upper_limit (int, optional): 选取内容最多多少字. Defaults to 100.

    Returns:
        str: 需要记忆的内容
    """
    length = len(texts)
    while True:
        start = random.randint(0, length-paragraphs)
        text = texts[start: start + random.randint(1, paragraphs+1)]
        text = " ".join(text)
        text_length = len(text)
        if upper_limit >= text_length >= lower_limit:
            break
    return text

# paras = select_sentence(texts, 2)
# print(paras)

### 计算相似度 (只计算中文字符的相似度) 并将相似度结果写入 log
这里面计算方法是有一点问题的 不过影响不大 参考参考就行

In [4]:
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese 

def compare_similarity(paras, words):
    memories[paras] = memories.get(paras, 0) + 1
    recall_count = memories[paras]
    print(f"第{recall_count}次回忆")
    paras_chinese = find_chinese(paras)
    words_chinese = find_chinese(words)
    
    # 计算正确词汇和错误词汇的个数
    count = 0
    wrong = 0
    wrong_words = []
    for word in words_chinese:
        if word in paras_chinese:
            count += 1
        else:
            wrong += 1
            wrong_words.append(word)
    
    # 计算遗漏的词汇和个数
    lack_count = 0
    lack_words = []
    for word in paras_chinese:
        if word not in words_chinese:
            lack_count += 1
            lack_words.append(word)
            
    lack_words = " ".join(lack_words)
    wrong_words = " ".join(wrong_words)
    accuracy = round(count / len(paras_chinese), 3)
    precision = round(count / len(words_chinese), 3)
    print("回忆内容:", words)
    print("正确答案:", paras)
    print(f"回忆覆盖率: {accuracy} ({count}/{len(paras_chinese)}), 回忆准确率: {precision} ({count}/{len(words_chinese)}), \n回忆正确字数: {count} ({len(words_chinese)}) 回忆错误字数: {wrong} ({len(paras_chinese)})")
    print("错误列表:", wrong_words)
    print("遗漏列表:", lack_words)
    with open("log.csv", "a+") as f:
        f.write(f'"{paras}", "{words}", {recall_count}, {accuracy}, {precision}, "{lack_words}", "{wrong_words}", {len(paras_chinese)}\n')
    with open("log_readable.txt", "a+") as f:
        f.write(f'{paras}\n')
        f.write(f'{words}\n')
        f.write(f"回忆覆盖率: {accuracy} ({count}/{len(paras_chinese)}), 回忆准确率: {precision} ({count}/{len(words_chinese)}), \n回忆正确字数: {count} ({len(words_chinese)}) 回忆错误字数: {wrong} ({len(paras_chinese)})\n\n")

# compare_similarity(paras, "要从眼前《人生》所造成的暖融融的气氛中，再一次踏进冰天雪地去进行一次看不见前 退堂的鼓声")

### 记忆内容 记忆后请折叠该单元格

In [5]:
paras = select_sentence(texts, paragraphs=3, lower_limit=90, upper_limit=100)
print(paras)

大多数特征都是“双刃剑”，可能带来好处和害处。 特征越极端，它可能带来的好处或害处就越极端。 例如，一个有高度创造性和目标导向性的人擅长设想新主意，但可能会低估日常生活细节的价值，而后者也很重要。


In [22]:
words = input("请输入回忆内容:")
compare_similarity(paras, words)

recall count 2
回忆内容:  如此一来每个人都清楚这是极端情况 在这一时期，即使是寻常做法，也可能被临时终止，要当心可能有人因为机构的利益而放弃创意择优，如果这种想法占了上风创意择优就会被弱化。
正确答案: 如此一来，每个人都清楚这是极端情况，而且在这一时期就算寻常做法也有可能会被临时中止。 b.当心可能有人会提出为了“机构的利益”而临时放弃创意择优。 如果这种想法占了上风，创意择优就会被弱化。
回忆覆盖率: 0.855 (71/83), 回忆准确率: 0.934 (71/76), 
回忆正确字数: 71 (76) 回忆错误字数: 5 (83)
错误列表: 即 使 终 要 因
遗漏列表: 且 算 中 提 出


In [ ]:
data = pd.read_csv("log.csv", error_bad_lines=False, warn_bad_lines=False)

In [47]:
data.groupby("回忆次数")[["覆盖率", "准确率"]].describe()

覆盖率                                                               准确率  \
      count      mean       std    min     25%     50%      75%    max  count   
回忆次数                                                                            
1     207.0  0.807507  0.160472  0.000  0.7500  0.8410  0.90650  1.059  207.0   
2     159.0  0.901541  0.095933  0.451  0.8500  0.9070  0.97900  1.108  159.0   
3      58.0  0.914707  0.076611  0.678  0.8785  0.9150  0.97300  1.071   58.0   
4      14.0  0.920643  0.091089  0.722  0.8750  0.9325  0.99475  1.036   14.0   
5       3.0  0.950000  0.039737  0.908  0.9315  0.9550  0.97100  0.987    3.0   
7       1.0  0.818000       NaN  0.818  0.8180  0.8180  0.81800  0.818    1.0   

                                                                 
          mean       std    min      25%    50%      75%    max  
回忆次数                                                             
1     0.894145  0.128251  0.000  0.86950  0.923  0.94750  1.000  
2     0.941767  0.050444  0.745  0.91350  0.951  0.97800  1.000  
3     0.936534  0.047875  0.787  0.90900  0.948  0.97150  1.000  
4     0.941643  0.066065  0.736  0.92725  0.962  0.97875  1.000  
5     0.959667  0.018610  0.940  0.95100  0.962  0.96950  0.977  
7     0.900000       NaN  0.900  0.90000  0.900  0.90000  0.900

In [48]:
data

,文本,回忆,回忆次数,覆盖率,准确率,遗漏,错字,总字数
0,要从眼前《人生》所造成的暖融融的气氛中，再一次踏进冰天雪地去进行一次看不见前途的远征，耳边就...,"""要从眼前《人生》所造成的暖融融的气氛中，再一次踏进冰天雪地去进行一次看不见前 退堂的鼓声""",1,0.784,1.000,"""途 远 征 耳 边 就 时 响 起""","""""",51
1,要从眼前《人生》所造成的暖融融的气氛中，再一次踏进冰天雪地去进行一次看不见前途的远征，耳边就...,"""要从目前暖融融的人生所造成的气氛中 再一次踏进冰天雪地去进行一场看不见前途的远征 耳边就...",2,0.902,0.885,"""眼 时 响 起""","""目 场 是 是 传 来""",51
2,要从眼前《人生》所造成的暖融融的气氛中，再一次踏进冰天雪地去进行一次看不见前途的远征，耳边就...,"""要从眼前人生所造成的的暖融融的气氛中 再一次踏进冰天雪地进行一场看不见前途的远征 而且就...",3,0.941,0.941,"""去 耳 边""","""场 而 且""",51
3,由此，这劳动就是平凡的劳动，而不应该有什么了不起的感觉；由此，你写平凡的世界，你也就是这平凡...,"""由此你写平凡的劳动 就是平凡的劳动 没有什么特殊的感觉; 由此你写平凡的劳动 那你就是平...",1,0.756,0.907,"""这 应 该 了 起 世 界 也 这 世 界 中 员 月 个 常 常 子 个 子""","""没 那 天 没 别 之 处""",90
4,由此，这劳动就是平凡的劳动，而不应该有什么了不起的感觉；由此，你写平凡的世界，你也就是这平凡...,"""由此 你写平凡的劳动 就是平凡的劳动 没有什么了不起的感觉; 由此 你写平凡的世界 ...",2,0.767,0.908,"""这 而 也 这 中 员 而 个 常 常 子 而 个 殊 子""","""没 那 天 没 别 之 处""",90
...,...,...,...,...,...,...,...,...
437,我一直对这段关系为何会走到这一步感到十分困惑，她单方面就给我定了罪状，然后不给我辩白机会，就...,"""我一直对这段关系为何会走到这一步，感到十分困惑，他单方面就给我定了罪状，然后不给我辩白的...",1,0.906,0.917,"""她 断 只 在 夜 息 自 己""","""他 删 除 联 上 他 短""",85
438,往日的柔情犹如一场突如其来的暖风，风过之后，剩下的只有凄凉潮湿的空气。 孤独之后仍旧是孤独。...,"""往日的柔情像一场突如其来的暖风 风过之后，就只剩凄凉和潮湿的空气 孤独之后仍是孤独 我以...",1,0.857,0.889,"""犹 下 有 旧 在 似 乎 际 么""","""像 就 和 以 为 将 时 候 样""",84
439,往日的柔情犹如一场突如其来的暖风，风过之后，剩下的只有凄凉潮湿的空气。 孤独之后仍旧是孤独。...,"""往日的柔情像一场突如其来的暖风 风过之后，只剩潮湿凄凉的空气 孤独之后仍旧是孤独，就当黎...",2,0.905,0.916,"""犹 下 有 在 似 乎 际""","""像 就 当 就 前 关 系""",84
440,只不过，多数帝国精英仍然一心相信，自己是为了所有帝国子民的整体福利而努力。 在中国的统治阶级...,"""只不过多数帝国精英仍然一心相信 自己是在为帝国所有人民的整体福利而努力 在中国统治阶级的...",1,0.890,0.859,"""了 子 个 邻 该 被 广 传 华 夏 文 化""","""人 以 理 披 万 物 这 批 万 物 汇 集""",82


In [49]:
print("回忆字数", data["回忆"].apply(len).sum())
print("记忆字数", data["文本"].apply(len).sum())
print("去重后记忆字数", data.drop_duplicates("文本")["文本"].apply(len).sum())

回忆字数 26917
记忆字数 28888
去重后记忆字数 13535
